In [64]:
from pdf2image import convert_from_path
pages = convert_from_path(r'docs\patient_details\pd_2.pdf', poppler_path=r'C:\poppler-25.12.0\Library\bin')

In [65]:
pages

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1500x2000>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1992x2000>]

In [66]:
pages[0].show()

In [67]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text = pytesseract.image_to_string(pages[0], lang = 'eng')
print(text)

a

Patient Medical Record

Patient Information
Jerry Lucas

(279) 920-8204
4218 Wheeler Ridge Dr

Buffalo, New York, 14201
United States

In Case of Emergency

Birth Date
May 2 1998

Weight:
57

Height:
170

Joe Lucas

Home phone

General Medical History

4218 Wheeler Ridge Dr
Buffalo, New York, 14201
United States

Work phone

Chicken Pox (Varicelia):

IMMUNE

Have you had the Hepatitis B vaccination?

Measles:

NOT IMMUNE

any Medical Problems (asthma, seizures, headaches):




In [68]:
import numpy as np
import cv2
from PIL import Image
def preprocess_image(img):
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
    resized_image = cv2.resize(gray, None, fx = 1.5, fy = 1.5, interpolation = cv2.INTER_LINEAR)
    processed_image = cv2.adaptiveThreshold(
        resized_image,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        61,
        20
    )
    return processed_image

In [69]:
img = preprocess_image(pages[0])
from PIL import Image
Image.fromarray(img).show()
text = pytesseract.image_to_string(img, lang = 'eng')
print(text)

17/12/2020

Patient Medical Record

Patient Information Birth Date
Jerry Lucas May 2 1998
(279) 920-8204 Weight:
4218 Wheeler Ridge Dr 57

Buffalo, New York, 14201

United States Height:

170

In Case of Emergency
ee

Joe Lucas 4218 Wheeler Ridge Dr
Buffalo, New York, 14201
Home phone . United States
Work phone

General Medical History

Chicken Pox (Varicelia): Measles:

IMMUNE NOT IMMUNE

Have you had the Hepatitis B vaccination?

“Yes

List any Medical Problems (asthma, seizures, headaches):
N/A



<h2>Extract Name</h2>


In [70]:
import re
pattern = r'Patient Information(.*?)\(\d{3}\)'
matches = re.findall(pattern, text, flags=re.DOTALL)
matches

[' Birth Date\nJerry Lucas May 2 1998\n']

In [71]:
matches[0].strip()

'Birth Date\nJerry Lucas May 2 1998'

In [72]:
match = matches[0].replace("Birth Date", "").strip()
match

'Jerry Lucas May 2 1998'

In [73]:
pattern = r'((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
date_matches = re.findall(pattern, match)
date = date_matches[0][0]
date

'May 2 1998'

In [74]:
match.replace(date, '').strip()

'Jerry Lucas'

In [75]:
def remove_noise_from_name(name):
    name = name.replace('Birth Date', '').strip()
    date_pattern = r'((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
    date_matches = re.findall(date_pattern, name)

    if date_matches:
        date = date_matches[0][0]
        name = name.replace(date, '').strip()

    return name

In [76]:
name = ' Birth Date\nKathy Crawford May 6 1972\n'
name = remove_noise_from_name(name)
name

'Kathy Crawford'

<h2>Extract Phone</h2>


In [77]:
pattern = r'Patient Information(.*?)(\(\d{3}\) \d{3}-\d{4})'
matches = re.findall(pattern, text, flags=re.DOTALL)
matches

[(' Birth Date\nJerry Lucas May 2 1998\n', '(279) 920-8204')]

In [78]:
matches[0][1]

'(279) 920-8204'

<h2>Extract Vaccine</h2>


In [79]:
pattern = r'Have you had the Hepatitis B vaccination\?.*(Yes|No)'
matches = re.findall(pattern, text, flags=re.DOTALL)
matches

['Yes']

<h2>Extract Medical Problems</h2>


In [80]:
pattern = r'List any Medical Problems \(asthma, seizures, headaches\)\:(.*)'
matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'N/A'